# SPARK SCENARIOS 

In [1]:
from pyspark.sql import SparkSession,DataFrame
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [24]:
spark=SparkSession.builder.\
appName("spark_scenarios").\
master("yarn").\
config("spark.port.maxRetries","300").\
config('spark.default.parallelism',10).\
enableHiveSupport().\
getOrCreate()

In [25]:
spark.sparkContext.applicationId

'application_1615503744169_2064'

In [16]:
# df=spark.read.options(recursiveFileLookup=True,inferSchema=True,delimiter="\t").\
# csv("/user/itv452844/test_data/recursive_read/hr_db")

In [26]:
df=spark.range(10)

In [27]:
df.rdd.getNumPartitions()

10

In [28]:
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [29]:
df.count()

10

In [ ]:
spark.conf.get("spark.sql.adaptive.skewJoin.skewedPartitionThresholdInBytes")

In [ ]:
# spark.read.table("select ")

In [ ]:
schema=StructType([StructField('CTY_CDE',StringType()),StructField('CTY_NAME',StringType()),StructField('CNT',IntegerType())])
df=spark.read.csv('/public/hr_db/countries',sep='\t',schema=schema)

In [ ]:
df.count()

In [ ]:
df.write.saveAsTable("pathirippilly.COUNTRIES")

In [ ]:
# %config Completer.use_jedi = False
# !pip install --upgrade jedi==0.17.2
!hadoop fs -ls /public/hr_db/coun

1. Recursively read file from folders

a. Using recursiveFileLookup option in spark 3.0 +

In [ ]:
df=spark.read.options(recursiveFileLookup=True,inferSchema=True,delimiter="\t").\
csv("/user/itv452844/test_data/recursive_read/hr_db")

In [ ]:
df

In [ ]:
# ! hadoop fs -cat /user/itv452844/test_data/recursive_read/hr_db/part1/part-m-00000

2. Replace Values from an arrayType Column

In [ ]:
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,ArrayType,IntegerType,StringType

* create test dataframe with arrayType column

In [ ]:
schema=StructType([StructField('id',IntegerType()),StructField('val',ArrayType(StringType()))])

row={'id':1,'val':['null','2','null','2','4']}

df=spark.createDataFrame([Row(**row)],schema=schema)

df.toPandas() # input dataframe

In [ ]:
df.createOrReplaceTempView("dfTable") # regestering as a temp view for querying

In [ ]:
# query for replacing elements from a arrayType Column
qry="""
select 
id,
split(replace(concat_ws(',',val),'null',''),',') as val
from
dfTable
"""
spark.sql(qry)

3. remove columns with all values as null

In [ ]:
schema=StructType([StructField('col1',IntegerType()),StructField('col2',IntegerType()),StructField('col3',IntegerType())])
data=[Row(**{'col1':1,'col2':None,'col3':1}),
Row(**{'col1':1,'col2':None,'col3':1}),
Row(**{'col1':1,'col2':None,'col3':None}),
Row(**{'col1':1,'col2':None,'col3':1})]

In [ ]:
df=spark.createDataFrame(data,schema=schema)

In [ ]:
df.show()

In [ ]:
def getDropList(df):
    
    colsToDrop=df.rdd.\
    map(lambda x : x.asDict()).\
    flatMap(lambda x : list(x.items())).\
    groupByKey().\
    mapValues(set).\
    filter(lambda x : list(x[1])[0]==None and len(x[1])==1).\
    collect()
    
    if len(colsToDrop)>0:
        return [x[0] for x in colsToDrop]
        

In [ ]:
df.drop(*getDropList(df))